# Proyecto análisis de conversación de mensajería

## Cargar datos
- Exportar una conversación personal o grupal de Whastapp o Telegram en un formato adecuado (JSON, CSV, etc), no exportar audios, archivos multimedia, etc
- Importar archivo y convertirlo en dataframe

## Conocer los datos

- Observar que tipo de datos tiene el dataframe y que tipo de valores tiene
- Explicar que significa cada campo y que tipo de acciones quedan registradas

## Datos faltantes

- Analizar de manera visual y no visual si existen datos faltantes

- Tratar datos faltantes en el caso que los haya
- Explicar qué método se utilizó y por qué

## Outliers

- Analizar de manera visual y no visual si existen valores atípicos para cada valor no categórico

- Si existen valores atípicos, ¿estos son causados por errores en la carga de datos o por otra razón?. Explicar la razón